<a href="https://colab.research.google.com/github/shafiqulrehman/AlzheimerCode/blob/main/DARWIN_Different_Ensemble_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Applying different Ensemble models**
1. Majority voting
2. Bagging (Bootstrap Aggregating)
3. Boosting
4. Gradient Boosted Decision Trees (GBDT)
5. Bayesian Model Averaging
6. Bootstrap Aggregating for Time Series (BATS)
7. Stacking


In [ ]:
########### Ensemble model using majority voting  #########

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score

# Read the dataset from CSV
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Preprocess the dataset
def preprocess_data(df):
    # Assuming the last column is the class label
    X = df.iloc[:, :-1]  # Features
    y = df.iloc[:, -1]   # Class labels

    # Convert class labels to numerical values (if needed)
    y = y.map({'H': 0, 'P': 1})

    return X, y

# Apply Recursive Feature Elimination (RFE) and train classifier
def rfe_and_train(X_train, y_train, k, classifier):
    model = classifier(random_state=42)
    rfe = RFE(estimator=model, n_features_to_select=k)
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    model.fit(X_train_rfe, y_train)
    return model, rfe

# Apply SelectKBest with ANOVA and train classifier
def selectkbest_and_train(X_train, y_train, k, classifier):
    skb = SelectKBest(f_classif, k=k)
    X_train_skb = skb.fit_transform(X_train, y_train)
    model = classifier()
    model.fit(X_train_skb, y_train)
    return model, skb

# Combine predictions using majority voting
def combine_predictions(predictions):
    # Use mode to get the most common prediction for each sample
    majority_votes = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
    return majority_votes

# Evaluate the model on test data
def evaluate_model(model, X_test, y_test, selector):
    X_test_selected = selector.transform(X_test)
    y_pred = model.predict(X_test_selected)
    return y_pred

# Calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    precision = precision_score(y_true, y_pred)
    tpr = sensitivity
    fpr = 1 - specificity
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc

# Main function
def main():
    # Path to your CSV file
    file_path = '/content/data_null_outlier_free_normalized_corr_90.csv'

    # Read the dataset
    df = read_csv(file_path)

    # Preprocess the data
    X, y = preprocess_data(df)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set the value of k for Extra Trees classifier
    k_et = 200

    # Apply RFE and train the Extra Trees classifier
    model_et, rfe_et = rfe_and_train(X_train, y_train, k_et, ExtraTreesClassifier)

    # Set the value of k for Random Forest classifier
    k_rf = 250

    # Apply RFE and train the Random Forest classifier
    model_rf, rfe_rf = rfe_and_train(X_train, y_train, k_rf, RandomForestClassifier)

    # Set the value of k for Logistic Regression classifier
    k_lr = 50

    # Apply RFE and train the Logistic Regression classifier
    model_lr, rfe_lr = rfe_and_train(X_train, y_train, k_lr, LogisticRegression)

    # Set the value of k for Gaussian Naive Bayes classifier
    k_gnb = 320

    # Apply SelectKBest with ANOVA and train the Gaussian Naive Bayes classifier
    model_gnb, skb_gnb = selectkbest_and_train(X_train, y_train, k_gnb, GaussianNB)

    # Set the value of k for SVM classifier
    k_svm = 150

    # Apply SelectKBest with ANOVA and train the SVM classifier
    model_svm, skb_svm = selectkbest_and_train(X_train, y_train, k_svm, SVC)

    # Set the value of k for MLP classifier
    k_mlp = 290

    # Apply SelectKBest with ANOVA and train the MLP classifier
    model_mlp, skb_mlp = selectkbest_and_train(X_train, y_train, k_mlp, MLPClassifier)

    # Set the value of k for XGB classifier
    k_xgb = 60

    # Apply RFE and train the XGB classifier
    model_xgb, rfe_xgb = rfe_and_train(X_train, y_train, k_xgb, XGBClassifier)

    # Evaluate each model on the test set
    predictions_et = evaluate_model(model_et, X_test, y_test, rfe_et)
    predictions_rf = evaluate_model(model_rf, X_test, y_test, rfe_rf)
    predictions_lr = evaluate_model(model_lr, X_test, y_test, rfe_lr)
    predictions_gnb = evaluate_model(model_gnb, X_test, y_test, skb_gnb)
    predictions_svm = evaluate_model(model_svm, X_test, y_test, skb_svm)
    predictions_mlp = evaluate_model(model_mlp, X_test, y_test, skb_mlp)
    predictions_xgb = evaluate_model(model_xgb, X_test, y_test, rfe_xgb)

    # Combine predictions using majority voting
    combined_predictions = combine_predictions([predictions_et, predictions_rf, predictions_svm, predictions_xgb, predictions_mlp, predictions_gnb  , predictions_lr ])
    # predictions_xgb, predictions_mlp,, predictions_gnb  , predictions_lr
    # Calculate evaluation metrics for the combined predictions
    acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc = calculate_metrics(y_test, combined_predictions)

    # Print the results
    print(f'Accuracy: {acc:.4f}')
    print(f'Sensitivity (Recall): {sensitivity:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'TPR (True Positive Rate): {tpr:.4f}')
    print(f'FPR (False Positive Rate): {fpr:.4f}')
    print(f'F1-score: {f1:.4f}')
    print(f'MCC (Matthews Correlation Coefficient): {mcc:.4f}')
    print(f'Cohen\'s Kappa: {kappa:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')

if __name__ == "__main__":
    main()


Accuracy: 0.9143
Sensitivity (Recall): 0.9000
Specificity: 0.9333
Precision: 0.9474
TPR (True Positive Rate): 0.9000
FPR (False Positive Rate): 0.0667
F1-score: 0.9231
MCC (Matthews Correlation Coefficient): 0.8278
Cohen's Kappa: 0.8264
AUC-ROC: 0.9167


In [ ]:
########### Bagging Ensemble Model #########

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score

# Read the dataset from CSV
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Preprocess the dataset
def preprocess_data(df):
    # Assuming the last column is the class label
    X = df.iloc[:, :-1]  # Features
    y = df.iloc[:, -1]   # Class labels

    # Convert class labels to numerical values (if needed)
    y = y.map({'H': 0, 'P': 1})

    return X, y

# Apply Recursive Feature Elimination (RFE) and train classifier
def rfe_and_train(X_train, y_train, k, classifier):
    model = classifier(random_state=42)
    rfe = RFE(estimator=model, n_features_to_select=k)
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    model.fit(X_train_rfe, y_train)
    return model, rfe

# Apply SelectKBest with ANOVA and train classifier
def selectkbest_and_train(X_train, y_train, k, classifier):
    skb = SelectKBest(f_classif, k=k)
    X_train_skb = skb.fit_transform(X_train, y_train)
    model = classifier()
    model.fit(X_train_skb, y_train)
    return model, skb

# Evaluate the model on test data
def evaluate_model(model, X_test, y_test, selector):
    X_test_selected = selector.transform(X_test)
    y_pred = model.predict(X_test_selected)
    return y_pred

# Calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    precision = precision_score(y_true, y_pred)
    tpr = sensitivity
    fpr = 1 - specificity
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc

# Main function
def main():
    # Path to your CSV file
    file_path = '/content/data_null_outlier_free_normalized_corr_90.csv'

    # Read the dataset
    df = read_csv(file_path)

    # Preprocess the data
    X, y = preprocess_data(df)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set the value of k for Extra Trees classifier
    k_et = 200

    # Apply RFE and train the Extra Trees classifier
    model_et, rfe_et = rfe_and_train(X_train, y_train, k_et, ExtraTreesClassifier)

    # Set the value of k for Random Forest classifier
    k_rf = 250

    # Apply RFE and train the Random Forest classifier
    model_rf, rfe_rf = rfe_and_train(X_train, y_train, k_rf, RandomForestClassifier)

    # Set the value of k for Logistic Regression classifier
    k_lr = 50

    # Apply RFE and train the Logistic Regression classifier
    model_lr, rfe_lr = rfe_and_train(X_train, y_train, k_lr, LogisticRegression)

    # Set the value of k for Gaussian Naive Bayes classifier
    k_gnb = 320

    # Apply SelectKBest with ANOVA and train the Gaussian Naive Bayes classifier
    model_gnb, skb_gnb = selectkbest_and_train(X_train, y_train, k_gnb, GaussianNB)

    # Set the value of k for SVM classifier
    k_svm = 150

    # Apply SelectKBest with ANOVA and train the SVM classifier
    model_svm, skb_svm = selectkbest_and_train(X_train, y_train, k_svm, SVC)

    # Set the value of k for MLP classifier
    k_mlp = 290

    # Apply SelectKBest with ANOVA and train the MLP classifier
    model_mlp, skb_mlp = selectkbest_and_train(X_train, y_train, k_mlp, MLPClassifier)

    # Set the value of k for XGB classifier
    k_xgb = 60

    # Apply RFE and train the XGB classifier
    model_xgb, rfe_xgb = rfe_and_train(X_train, y_train, k_xgb, XGBClassifier)

    # Initialize Bagging classifier for each base model
    base_models = [
        ('ET', model_et),
        ('RF', model_rf),
        ('LR', model_lr),
        ('GNB', model_gnb),
        ('SVM', model_svm),
        ('MLP', model_mlp),
        ('XGB', model_xgb)
    ]
    bagging_ensemble = BaggingClassifier(base_estimator=None, n_estimators=10, random_state=42)

    # Fit the Bagging ensemble
    bagging_ensemble.fit(X_train, y_train)

    # Evaluate the Bagging ensemble on the test set
    y_pred_bagging = bagging_ensemble.predict(X_test)

    # Calculate evaluation metrics for the Bagging ensemble
    acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc = calculate_metrics(y_test, y_pred_bagging)

    # Print the results
    print(f'Accuracy: {acc:.4f}')
    print(f'Sensitivity (Recall): {sensitivity:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'TPR (True Positive Rate): {tpr:.4f}')
    print(f'FPR (False Positive Rate): {fpr:.4f}')
    print(f'F1-score: {f1:.4f}')
    print(f'MCC (Matthews Correlation Coefficient): {mcc:.4f}')
    print(f'Cohen\'s Kappa: {kappa:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')

if __name__ == "__main__":
    main()


Accuracy: 0.8286
Sensitivity (Recall): 0.7500
Specificity: 0.9333
Precision: 0.9375
TPR (True Positive Rate): 0.7500
FPR (False Positive Rate): 0.0667
F1-score: 0.8333
MCC (Matthews Correlation Coefficient): 0.6788
Cohen's Kappa: 0.6613
AUC-ROC: 0.8417


In [ ]:
########### Boosting Ensemble Model #########

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score

# Read the dataset from CSV
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Preprocess the dataset
def preprocess_data(df):
    # Assuming the last column is the class label
    X = df.iloc[:, :-1]  # Features
    y = df.iloc[:, -1]   # Class labels

    # Convert class labels to numerical values (if needed)
    y = y.map({'H': 0, 'P': 1})

    return X, y

# Apply Recursive Feature Elimination (RFE) and train classifier
def rfe_and_train(X_train, y_train, k, classifier):
    model = classifier(random_state=42)
    rfe = RFE(estimator=model, n_features_to_select=k)
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    model.fit(X_train_rfe, y_train)
    return model, rfe

# Apply SelectKBest with ANOVA and train classifier
def selectkbest_and_train(X_train, y_train, k, classifier):
    skb = SelectKBest(f_classif, k=k)
    X_train_skb = skb.fit_transform(X_train, y_train)
    model = classifier()
    model.fit(X_train_skb, y_train)
    return model, skb

# Evaluate the model on test data
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return y_pred

# Calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    precision = precision_score(y_true, y_pred)
    tpr = sensitivity
    fpr = 1 - specificity
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc

# Main function
def main():
    # Path to your CSV file
    file_path = '/content/data_null_outlier_free_normalized_corr_90.csv'

    # Read the dataset
    df = read_csv(file_path)

    # Preprocess the data
    X, y = preprocess_data(df)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set the value of k for Extra Trees classifier
    k_et = 200

    # Apply RFE and train the Extra Trees classifier
    model_et, rfe_et = rfe_and_train(X_train, y_train, k_et, ExtraTreesClassifier)

    # Set the value of k for Random Forest classifier
    k_rf = 250

    # Apply RFE and train the Random Forest classifier
    model_rf, rfe_rf = rfe_and_train(X_train, y_train, k_rf, RandomForestClassifier)

    # Set the value of k for Logistic Regression classifier
    k_lr = 50

    # Apply RFE and train the Logistic Regression classifier
    model_lr, rfe_lr = rfe_and_train(X_train, y_train, k_lr, LogisticRegression)

    # Set the value of k for Gaussian Naive Bayes classifier
    k_gnb = 320

    # Apply SelectKBest with ANOVA and train the Gaussian Naive Bayes classifier
    model_gnb, skb_gnb = selectkbest_and_train(X_train, y_train, k_gnb, GaussianNB)

    # Set the value of k for Decision Tree classifier
    k_dt = 60

    # Apply RFE and train the Decision Tree classifier
    model_dt, rfe_dt = rfe_and_train(X_train, y_train, k_dt, DecisionTreeClassifier)

    # Initialize AdaBoost classifier
    base_estimators = [model_et, model_rf, model_lr, model_gnb, model_dt]
    adaboost_ensemble = AdaBoostClassifier(base_estimator=None, n_estimators=50, random_state=42)

    # Fit the AdaBoost ensemble
    adaboost_ensemble.fit(X_train, y_train)

    # Evaluate the AdaBoost ensemble on the test set
    y_pred_adaboost = evaluate_model(adaboost_ensemble, X_test, y_test)

    # Calculate evaluation metrics for the AdaBoost ensemble
    acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc = calculate_metrics(y_test, y_pred_adaboost)

    # Print the results
    print(f'Accuracy: {acc:.4f}')
    print(f'Sensitivity (Recall): {sensitivity:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'TPR (True Positive Rate): {tpr:.4f}')
    print(f'FPR (False Positive Rate): {fpr:.4f}')
    print(f'F1-score: {f1:.4f}')
    print(f'MCC (Matthews Correlation Coefficient): {mcc:.4f}')
    print(f'Cohen\'s Kappa: {kappa:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')

if __name__ == "__main__":
    main()


In [ ]:
######## Ensemble model using Stacking ##########

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score
from sklearn.ensemble import StackingClassifier

# Read the dataset from CSV
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Preprocess the dataset
def preprocess_data(df):
    # Assuming the last column is the class label
    X = df.iloc[:, :-1]  # Features
    y = df.iloc[:, -1]   # Class labels

    # Convert class labels to numerical values (if needed)
    y = y.map({'H': 0, 'P': 1})

    return X, y

# Apply Recursive Feature Elimination (RFE) and train classifier
def rfe_and_train(X_train, y_train, k, classifier):
    model = classifier(random_state=42)
    rfe = RFE(estimator=model, n_features_to_select=k)
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    model.fit(X_train_rfe, y_train)
    return model, rfe

# Apply SelectKBest with ANOVA and train classifier
def selectkbest_and_train(X_train, y_train, k, classifier):
    skb = SelectKBest(f_classif, k=k)
    X_train_skb = skb.fit_transform(X_train, y_train)
    model = classifier()
    model.fit(X_train_skb, y_train)
    return model, skb

# Combine predictions using majority voting
def combine_predictions(predictions):
    # Use mode to get the most common prediction for each sample
    majority_votes = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
    return majority_votes

# Evaluate the model on test data
def evaluate_model(model, X_test, y_test, selector):
    X_test_selected = selector.transform(X_test)
    y_pred = model.predict(X_test_selected)
    return y_pred

# Calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    precision = precision_score(y_true, y_pred)
    tpr = sensitivity
    fpr = 1 - specificity
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc

# Main function
def main():
    # Path to your CSV file
    #file_path = '/content/data_null_outlier_free_normalized.csv'
    file_path = '/content/data_null_outlier_free_normalized_corr_90.csv'
    # Read the dataset
    df = read_csv(file_path)

    # Preprocess the data
    X, y = preprocess_data(df)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set the value of k for Extra Trees classifier
    k_et = 200

    # Apply RFE and train the Extra Trees classifier
    model_et, rfe_et = rfe_and_train(X_train, y_train, k_et, ExtraTreesClassifier)

    # Set the value of k for Random Forest classifier
    k_rf = 250

    # Apply RFE and train the Random Forest classifier
    model_rf, rfe_rf = rfe_and_train(X_train, y_train, k_rf, RandomForestClassifier)

    # Set the value of k for SVM classifier
    k_svm = 150

    # Apply SelectKBest with ANOVA and train the SVM classifier
    model_svm, skb_svm = selectkbest_and_train(X_train, y_train, k_svm, SVC)

    # Combine predictions from base-level models
    base_level_predictions = np.column_stack([evaluate_model(model_et, X_test, y_test, rfe_et),
                                              evaluate_model(model_rf, X_test, y_test, rfe_rf),
                                              evaluate_model(model_svm, X_test, y_test, skb_svm)])

    # Train the meta-label classifier (Logistic Regression)
    meta_classifier = LogisticRegression(random_state=42)
    stacking_model = StackingClassifier(estimators=[
        ('et', ExtraTreesClassifier(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svm', SVC(random_state=42)),
    ], final_estimator=meta_classifier)

    # Fit the stacking model on base-level predictions
    stacking_model.fit(base_level_predictions, y_test)

    # Make predictions using the stacking model
    stacking_predictions = stacking_model.predict(base_level_predictions)

    # Calculate evaluation metrics for the stacking predictions
    acc, sensitivity, specificity, precision, tpr, fpr, f1, mcc, kappa, auc_roc = calculate_metrics(y_test, stacking_predictions)

    # Print the results
    print('Results after stacking:')
    print(f'Accuracy: {acc:.4f}')
    print(f'Sensitivity (Recall): {sensitivity:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'TPR (True Positive Rate): {tpr:.4f}')
    print(f'FPR (False Positive Rate): {fpr:.4f}')
    print(f'F1-score: {f1:.4f}')
    print(f'MCC (Matthews Correlation Coefficient): {mcc:.4f}')
    print(f'Cohen\'s Kappa: {kappa:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')

if __name__ == "__main__":
    main()


Results after stacking:
Accuracy: 0.9714
Sensitivity (Recall): 0.9500
Specificity: 1.0000
Precision: 1.0000
TPR (True Positive Rate): 0.9500
FPR (False Positive Rate): 0.0000
F1-score: 0.9744
MCC (Matthews Correlation Coefficient): 0.9437
Cohen's Kappa: 0.9421
AUC-ROC: 0.9750
